# Vllm Setup (Using GLM-4 as an example)

### Step 1
install necessary dependency, especially [vllm](https://pypi.org/project/vllm/) and [transformers](https://pypi.org/project/transformers/)

using python 3.11+ and conda environment is recommended

```sh
conda create -n vllm python=3.11
pip install -r requirements.txt & pip install vllm transformers
```

### Step 2
By default, vLLM downloads model from HuggingFace. If you would like to use models from ModelScope in the following examples, please set the environment variable:

```sh
export VLLM_USE_MODELSCOPE=True
```

### Step 3 
use vLLM for offline batched inference of GLM-4-9B-Chat and run it in our modelscope-agent!

In [4]:
import os
import sys
sys.path.insert(0, '/your/path/to/modelscope_agent')
from modelscope_agent.agents.role_play import RolePlay 
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
model_name='THUDM/glm-4-9b-chat'

# you can also use local model by setting the "model_name" as the llm local path
# model_name="/path/to/model/glm-4-9b-chat" 

# create an vllm.LLM instance and set up the tokenizer and sampling_params
# detailed information can be found in https://docs.vllm.ai/en/latest/getting_started/quickstart.html#offline-batched-inference
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
llm = LLM(
    model=model_name,
    tensor_parallel_size=1,
    max_model_len=131072,
    trust_remote_code=True,
    enforce_eager=True,
)
sampling_params = SamplingParams(temperature=0.95, max_tokens=1024, stop_token_ids=[151329, 151336, 151338])


# setting up a llm config
llm_config = {'model': 'THUDM/glm-4-9b-chat',
              'model_server':'vllm',
              'tokenizer':tokenizer,
              'llm':llm,
              'sampling_params':sampling_params,
              'stream':True}
function_list = []
role_template = '你是一个agent小助手，你需要根据用户的要求来回答他们的问题'
bot = RolePlay(function_list=function_list,llm=llm_config, instruction=role_template)

response = bot.run('你好，请以李云龙的语气和我对话')
text = ''
for chunk in response:
    text += chunk
print(text)

2024-06-07 17:19:20,884 - modelscope - INFO - PyTorch version 2.3.0 Found.
2024-06-07 17:19:20,887 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-06-07 17:19:21,116 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-06-07 17:19:21,181 - modelscope - INFO - Loading done! Current index file version is 1.12.0, with md5 f25c671d28d2d28625a947bf48aaa515 and a total number of 964 components indexed
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 06-07 17:19:23 tokenizer.py:126] Using a slow tokenizer. This might cause a significant slowdown. Consider using a fast tokenizer instead.
INFO 06-07 17:19:28 model_runner.py:146] Loading model weights took 17.5635 GB
INFO 06-07 17:19:38 gpu_executor.py:83] # GPU blocks: 57488, # CPU blocks: 6553


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.36it/s, Generation Speed: 61.43 toks/s]
2024-06-07 17:19:40.993 - modelscope-agent - INFO -  | message: call vllm success, output: 
哈哈，嘿兄弟，诸葛同学，啥事呀？我这几天连轴转，忙得脚打后脑勺，啥风把你给吹来了？说吧，是不是又有啥新鲜事啊？
2024-06-07 17:19:40.994 - modelscope-agent - INFO -  | message: call llm 1 times output: 
哈哈，嘿兄弟，诸葛同学，啥事呀？我这几天连轴转，忙得脚打后脑勺，啥风把你给吹来了？说吧，是不是又有啥新鲜事啊？



哈哈，嘿兄弟，诸葛同学，啥事呀？我这几天连轴转，忙得脚打后脑勺，啥风把你给吹来了？说吧，是不是又有啥新鲜事啊？
